In [1]:
%pylab widget
from scipy import interpolate
from scipy import optimize
from numba import njit
import tqdm

Populating the interactive namespace from numpy and matplotlib


Our aim in this notebook is to reproduce [this](https://arxiv.org/pdf/1403.2259.pdf) paper.

In [2]:
nk = 200
kran = linspace(-pi,pi,nk,endpoint=False)
kx,ky = meshgrid(kran,kran)
kx = kx.flatten()
ky = ky.flatten()
gamma = -(1+exp(-1j*2*kx)+exp(-1j*(kx-ky))+exp(-1j*(kx+ky)))
H0k = zeros((len(gamma),2,2),dtype=complex)
H0k[:,0,1] = gamma
H0k[:,1,0] = conj(gamma)

In [3]:
def make_spectr(U,n1,n2):
    Hk = H0k.copy()
    Hk[:,0,0] = U*n1
    Hk[:,1,1] = U*n2
    return eigh(Hk)

In [4]:
@njit
def fFD(e,kbt=0.01):
    return 1/(exp(e/kbt)+1)
@njit
def dfFD(e,kbt=0.01):
    return -1/(kbt*cosh(e/kbt/2))
# pay respect to the jit god
fFD(rand(10,10),0.1);
dfFD(rand(10,10),0.1);

In [5]:
def scf_update(ns,U=1,ne=1,kbt=0.05,root=True):
    
    # unpack occupation numbers
    n1up,n1do,n2up,n2do = ns
    
    # get new spectrum and wavefunctions
    # vals_up[ik,i] the i-th eigenvalue of up electrons at the ik-th k point
    # vecs_up[ik,site,i] the i-th wavefunction of up electrons at the ik-th k point evaluated at site "site".
    vals_up,vecs_up = make_spectr(U,n1do,n2do)
    vals_do,vecs_do = make_spectr(U,n1up,n2up)

    # find new value for the chemical potential
    rho,eran = histogram(array([vals_up,vals_do]).flatten(),1000,density=True)
    rho = convolve(rho,hamming(10),mode='same')/5 # some additional smoothing
    NE = cumsum(rho)*diff(eran)[0]
    mu_from_ne = interpolate.interp1d(4*NE, eran[1:],kind='zero',fill_value="extrapolate")
    mu = float(mu_from_ne(ne))
    
    # find new occupation numbers
    n1up_new = (sum(abs(vecs_up[:,0,0])**2*fFD(vals_up[:,0]-mu,kbt))
               +sum(abs(vecs_up[:,0,1])**2*fFD(vals_up[:,1]-mu,kbt)))/nk**2

    n2up_new = (sum(abs(vecs_up[:,1,0])**2*fFD(vals_up[:,0]-mu,kbt))
               +sum(abs(vecs_up[:,1,1])**2*fFD(vals_up[:,1]-mu,kbt)))/nk**2

    n1do_new = (sum(abs(vecs_do[:,0,0])**2*fFD(vals_do[:,0]-mu,kbt))
               +sum(abs(vecs_do[:,0,1])**2*fFD(vals_do[:,1]-mu,kbt)))/nk**2

    n2do_new = (sum(abs(vecs_do[:,1,0])**2*fFD(vals_do[:,0]-mu,kbt))
               +sum(abs(vecs_do[:,1,1])**2*fFD(vals_do[:,1]-mu,kbt)))/nk**2

    if ((n1up > n1do) and (n1up_new < n1do_new)) or ((n1do > n1up) and (n1do_new < n1up_new) ):
        print('flip 1')
        n1up_new,n1do_new = n1do_new,n1up_new

    if ((n2up > n2do) and (n2up_new < n2do_new)) or ((n2do > n2up) and (n2do_new < n2up_new) ):
        print('flip 2')
        n2up_new,n2do_new = n2do_new,n2up_new

        
    ns_new = array([n1up_new,n1do_new,n2up_new,n2do_new])
    #print(sum(ns_new)-ne)
    ns_new = ns_new*ne/sum(ns_new) # force sum to be ne
        
    if root:    
        return ns_new-ns
    else:
        return ns_new


‘hybr’
‘lm’
‘broyden1’
‘broyden2’
‘anderson’
‘linearmixing’
‘diagbroyden’
‘excitingmixing’
‘krylov’
‘df-sane’ 

In [6]:
def get_etot(ns,U=1,kbt=0.05):

    # unpack occupation numbers
    n1up,n1do,n2up,n2do = ns
    
    # get new spectrum and wavefunctions
    # vals_up[ik,i] the i-th eigenvalue of up electrons at the ik-th k point
    # vecs_up[ik,site,i] the i-th wavefunction of up electrons at the ik-th k point evaluated at site "site".
    vals_up,vecs_up = make_spectr(U,n1do,n2do)
    vals_do,vecs_do = make_spectr(U,n1up,n2up)
    
    # find new value for the chemical potential
    rho,eran = histogram(array([vals_up,vals_do]).flatten(),1000,density=True)
    rho = convolve(rho,hamming(10),mode='same')/5 # some additional smoothing
    NE = cumsum(rho)*diff(eran)[0]
    mu_from_ne = interpolate.interp1d(4*NE, eran[1:],kind='zero',fill_value="extrapolate")
    mu = float(mu_from_ne(ne))
    
    etot=sum(vals_up[:,0]*fFD(vals_up[:,0]-mu,kbt))+\
         sum(vals_up[:,1]*fFD(vals_up[:,1]-mu,kbt))+\
         sum(vals_do[:,0]*fFD(vals_do[:,0]-mu,kbt))+\
         sum(vals_do[:,1]*fFD(vals_do[:,1]-mu,kbt))+\
        -U*(n1up*n1do+n2up*n2do)*len(vals_up)
    return etot

In [28]:
Uran=linspace(0.1,0.35,25)
kbt=0.005
ne = 1.6
delta= 1.5

n1up,n1do=ne/4+delta/4,ne/4-delta/4
n2up,n2do=ne/4+delta/4,ne/4-delta/4

ne = n1up+n1do+n2up+n2do # total number of particles in the unitcell 0..4
ns = array([n1up,n1do,n2up,n2do])

dat_ferro=[]
for ooU in tqdm.tqdm_notebook(Uran):
    scf_args = (1/ooU,ne,kbt,True)
    sol_root = optimize.root(scf_update,
                         ns,
                         args=scf_args,
                         method='broyden1',
                         options=dict(disp=False,fatol=1e-4))
    dat_ferro.append([sol_root.x,get_etot(sol_root.x,1/ooU,kbt)])

flip 1
flip 1
flip 1



In [31]:
Uran=linspace(0.1,0.35,25)
kbt=0.0125
ne = 1.6
delta= 1.5

n1up,n1do=ne/4+delta/4,ne/4-delta/4
n2up,n2do=ne/4-delta/4,ne/4+delta/4

ne = n1up+n1do+n2up+n2do # total number of particles in the unitcell 0..4
ns = array([n1up,n1do,n2up,n2do])

dat_antiferro=[]
for ooU in tqdm.tqdm_notebook(Uran):
    scf_args = (1/ooU,ne,kbt,True)
    sol_root = optimize.root(scf_update,
                         ns,
                         args=scf_args,
                         method='broyden1',
                         options=dict(disp=False,fatol=1e-4))
    dat_antiferro.append([sol_root.x,get_etot(sol_root.x,1/ooU,kbt)])

In [21]:
dat_para=[]
for ooU in tqdm.tqdm_notebook(Uran):
    dat_para.append([array([ne/4,ne/4,ne/4,ne/4]),get_etot([ne/4,ne/4,ne/4,ne/4],1/ooU,kbt)])

In [29]:
plt.close(1)
plt.figure(1)
subplot(211)
plot(Uran,[d[1]/nk**2 for d in dat_ferro],'r-',label='FM')
plot(Uran,[d[1]/nk**2 for d in dat_antiferro],'k--',label='AFM')
plot(Uran,[d[1]/nk**2 for d in dat_para],'b:',label='PM')
legend()
ylabel(r'$F$')
xlabel(r'$1/U$')
grid()
subplot(212)
plot(Uran,[abs(d[0][0]-d[0][1]) for d in dat_ferro],'r-',label='FM')
plot(Uran,[abs(d[0][0]-d[0][1]) for d in dat_antiferro],'k--',label='AFM')
plot(Uran,[abs(d[0][0]-d[0][1]) for d in dat_para],'b:',label='PM')
legend()
ylabel(r'$m$')
xlabel(r'$1/U$')
grid()
tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
Etot=array([[dat_ferro[i][1],dat_antiferro[i][1],dat_para[i][1]]for i in range(len(Uran))])/nk**2

In [40]:
fap=argmin(Etot,axis=1)

In [44]:
plt.close(2)
plt.figure(2)
plot(Uran[fap==0],ones_like(Uran[fap==0]),'o')
plot(Uran[fap==1],ones_like(Uran[fap==1]),'o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …